In [ ]:
# %load regressor.py
import numpy as np
import tensorflow as tf
import itertools
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn import preprocessing
import math as math
import os


tf.logging.set_verbosity(tf.logging.INFO)

# Learning rate for the model
LEARNING_RATE = 0.01

TRAIN_STEPS = 40000
BATCH_SIZE = 128


FILE_PATH = "../output/raceFeatures.csv"
# FILE_PATH = "../output/trainFeatures.csv"
# FILE_PATH = "../output/activitiesFeatures.csv"
# FILE_PATH = "../output/activitySetFeatures.csv"
PRED_PATH = "../output/predictions.csv"
SAVE_PATH = "temp"
COLUMNS = ["dist", "elev", "hilly", "cs", "atl", "ctl", "isRace", "avgVo2max", "time", "avgTrainPace"]
FEATURES = ["dist", "elev", "hilly", "cs", "atl", "ctl", "isRace", "avgVo2max", "avgTrainPace"]

LABEL = "time"

def plotStatistics():
	training_set, test_set, prediction_set = loadData()
	# training_set, test_set, prediction_set = normalize(training_set, test_set, prediction_set)
	dataset = pd.concat([training_set, test_set])
	# training_set.hist()


	# dataset.plot(kind='density', subplots=True, layout=(3,3), sharex=False)

	# pd.plotting.scatter_matrix(dataset)

	cax = plt.matshow(dataset.corr(), vmin=-1, vmax=1)
	plt.colorbar(cax)
	locs, labs = plt.xticks()
	plt.xticks(np.arange(len(COLUMNS)), COLUMNS)
	plt.yticks(np.arange(len(COLUMNS)), COLUMNS)


	# plt.scatter(dataset['dist'], dataset['CS'])


	plt.show()

def clearOldFiles():

	# filelist = [ f for f in os.listdir(SAVE_PATH)]
	# for f in filelist:
	# 	os.chmod(os.path.join(SAVE_PATH, f), 0o777)
	# 	os.remove(os.path.join(SAVE_PATH, f))
	if tf.gfile.Exists(SAVE_PATH):
   		tf.gfile.DeleteRecursively(SAVE_PATH) 

def normalize(train, test, pred):
	# label_train = train[LABEL]
	# label_test = test[LABEL]

	# mean, std = train[FEATURES].mean(axis=0), train[FEATURES].std(axis=0, ddof=0)
	# train = (train[FEATURES] - mean) /std
	# test = (test[FEATURES] - mean) / std
	# train = pd.concat([X_train, label_train], axis=1)
	# test = pd.concat([X_test, label_test], axis=1)

	# minmax_scale = preprocessing.MinMaxScaler(feature_range=(-1, 1)).fit(train[FEATURES])
	# train[FEATURES] = minmax_scale.transform(train[FEATURES])
	# test[FEATURES] = minmax_scale.transform(test[FEATURES])
	# pred[FEATURES] = minmax_scale.transform(pred[FEATURES])

	std_scale = preprocessing.StandardScaler().fit(train[FEATURES])
	train[FEATURES] = std_scale.transform(train[FEATURES])
	test[FEATURES] = std_scale.transform(test[FEATURES])
	pred[FEATURES] = std_scale.transform(pred[FEATURES])

	

	# print(train)
	return train, test, pred


def get_input_fn(data_set, num_epochs=None, shuffle=True):
	
	return tf.estimator.inputs.pandas_input_fn(x=pd.DataFrame({k: data_set[k].values for k in FEATURES}), 
  		y = pd.Series(data_set[LABEL].values), batch_size=BATCH_SIZE, num_epochs=num_epochs, shuffle=shuffle)




def loadData():
	

	trainData = pd.read_csv(FILE_PATH, skipinitialspace=True, skiprows=1, names=COLUMNS)
	testData = pd.read_csv(PRED_PATH, skipinitialspace=True, skiprows=1, names=COLUMNS)
	predData = pd.read_csv(PRED_PATH, skipinitialspace=True, skiprows=1, names=COLUMNS)
	# numRows = len(data.index)
	# train_rows = math.floor(numRows * 1)
	# test_rows = numRows - train_rows
	# training_set = data[:train_rows]
	# test_set = data[train_rows :train_rows + test_rows].reset_index(drop=True)
	# print(data)
	print('train size: ',len(trainData),' test size: ', len(testData))
	# print(training_set)
	# print(test_set)
	# print(predData)

	training_set = pd.DataFrame(trainData, columns=COLUMNS)
	test_set = pd.DataFrame(testData, columns=COLUMNS)
	prediction_set = pd.DataFrame(predData, columns=COLUMNS)

	return training_set, test_set, prediction_set


def model_fn(features, labels, mode, params):
	# Logic to do the following:
	# 1. Configure the model via TensorFlow operations
	# 2. Define the loss function for training/evaluation
	# 3. Define the training operation/optimizer
	# 4. Generate predictions
	# 5. Return predictions/loss/train_op/eval_metric_ops in EstimatorSpec object
 

	# Connect the first hidden layer to input layer
	feature_cols = [tf.feature_column.numeric_column(k) for k in FEATURES]
	input_layer = tf.feature_column.input_layer(features=features, feature_columns=feature_cols)


	# Connect the first hidden layer to second hidden layer with relu
	hidden_layer = tf.layers.dense(input_layer, 10, activation=tf.nn.relu, 
		kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(scale_l1=1.0, scale_l2=1.0), name='hidden_1')

	h1_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'hidden_1')
	tf.summary.histogram('kernel_1', h1_vars[0])
	tf.summary.histogram('bias_1', h1_vars[1])
	tf.summary.histogram('activation_1', hidden_layer)

	if mode == tf.estimator.ModeKeys.TRAIN:
		hidden_layer = tf.layers.dropout(hidden_layer, rate=0.3, name='dropout_1')
		tf.summary.scalar('dropout_1', tf.nn.zero_fraction(hidden_layer))



	# Connect the second hidden layer to first hidden layer with relu
	hidden_layer = tf.layers.dense(hidden_layer, 10, activation=tf.nn.relu, 
		kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(scale_l1=1.0, scale_l2=1.0), name='hidden_2')

	h2_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'hidden_2')
	tf.summary.histogram('kernel_2', h2_vars[0])
	tf.summary.histogram('bias_2', h2_vars[1])
	tf.summary.histogram('activation_2', hidden_layer)

	if mode == tf.estimator.ModeKeys.TRAIN:
		hidden_layer = tf.layers.dropout(hidden_layer, rate=0.3, name='dropout_2')
		tf.summary.scalar('dropout_2', tf.nn.zero_fraction(hidden_layer))



	# Connect the output layer to second hidden layer (no activation fn)
	output_layer = tf.layers.dense(hidden_layer, 1, name='output')

	# Reshape output layer to 1-dim Tensor to return predictions
	predictions = tf.reshape(output_layer, [-1])

	# Provide an estimator spec for `ModeKeys.PREDICT`.
	if mode == tf.estimator.ModeKeys.PREDICT:
		return tf.estimator.EstimatorSpec(mode=mode,predictions={LABEL: predictions})


	# Calculate loss using mean squared error
	loss = tf.losses.mean_squared_error(labels, predictions)

	reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
	loss = tf.add_n([loss] + reg_losses)

	
	tf.summary.scalar("reg_loss", reg_losses[0])
	tf.summary.scalar("train_error", loss)



	optimizer = tf.train.AdamOptimizer(learning_rate=params["learning_rate"])
	train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
	# grad= optimizer.compute_gradients(loss)
	# train_op = optimizer.apply_gradients(grad, global_step=tf.train.get_global_step())
	alpha_t = optimizer._lr * tf.sqrt(1-optimizer._beta2_power) / (1-optimizer._beta1_power)
	tf.summary.scalar("learning_rate", alpha_t)
	
	# for g, v in enumerate(grad):
	# 	tf.summary.scalar("gradient", g)

	# Calculate root mean squared error as additional eval metric
	eval_metric_ops = {
	  "rmse": tf.metrics.root_mean_squared_error(tf.cast(labels, tf.float64), tf.cast(predictions, tf.float64)),
	  "r" : tf.contrib.metrics.streaming_pearson_correlation(tf.cast(predictions, tf.float32), tf.cast(labels, tf.float32))
	}
	
	# Provide an estimator spec for `ModeKeys.EVAL` and `ModeKeys.TRAIN` modes.
	return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op, eval_metric_ops=eval_metric_ops)


	return EstimatorSpec(mode, predictions, loss, train_op, eval_metric_ops)


def main(unused_argv):
	predictionOnly = True;
	# clearOldFiles()

	training_set, test_set, prediction_set = loadData()

	# prediction_set = pd.DataFrame([(10000, 200, 0.2, 0.5, 44, 45, 1, 43, 38,3.448)], columns=COLUMNS)

	training_set, test_set, prediction_set = normalize(training_set, test_set, prediction_set)

	# Set model params
	model_params = {"learning_rate": LEARNING_RATE}

	# Instantiate Estimator
	nn = tf.estimator.Estimator(model_fn=model_fn, params=model_params, model_dir=SAVE_PATH)

	if not predictionOnly:
		train_input_fn = get_input_fn(training_set, num_epochs=None, shuffle=True)

		# Train
		nn.train(input_fn=train_input_fn, steps=TRAIN_STEPS)

		# Score accuracy
		test_input_fn = get_input_fn(test_set, num_epochs=1, shuffle=False)
		ev = nn.evaluate(input_fn=test_input_fn)
		print("Loss: %s" % ev['loss'])
		print("Root Mean Squared Error: %s" % ev["rmse"])
		print("R: %s" % ev["r"])


	# Print out predictions

	predict_input_fn = get_input_fn(prediction_set, num_epochs=1, shuffle=False)
	predictions = nn.predict(input_fn=predict_input_fn)
	pred = list()
	for i, p in enumerate(predictions):
		print("Prediction %s: %s" % (i + 1, p[LABEL]))
		pred.append(p[LABEL])
		print("Seconds away: %s" % ((prediction_set[LABEL][i] - p[LABEL]) * 60))

	
	# plt.scatter(prediction_set.loc[:,'dist'], pred)
	# plt.scatter(training_set.loc[:,'dist'], training_set.loc[:,'time'])
	# plt.show()



if __name__ == "__main__":
	tf.app.run()
	# plotStatistics()

Training consists of all available activities, then kmeans-selected athlete activities and lastly only the races.
Current Features: ["dist", "elev", "hilly", "cs", "atl", "ctl", "isRace", "avgVo2max", "time", "avgTrainPace"]

In [1]:
%run regressor.py

train size:  2573  test size:  5
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'temp', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into temp\model.ckpt.
INFO:tensorflow:loss = 5966.59, step = 1
INFO:tensorflow:global_step/sec: 492.351
INFO:tensorflow:loss = 345.965, step = 101 (0.203 sec)
INFO:tensorflow:global_step/sec: 533.327
INFO:tensorflow:loss = 212.51, step = 201 (0.203 sec)
INFO:tensorflow:global_step/sec: 492.304
INFO:tensorflow:loss = 200.305, step = 301 (0.187 sec)
INFO:tensorflow:global_step/sec: 533.322
INFO:tensorflow:loss = 197.177, step = 401 (0.203 sec)
INFO:tensorflow:global_step/sec: 492.308
INFO:tensorflow:loss = 135.538, step = 501 (0.188 sec)
INFO:tensorf

INFO:tensorflow:global_step/sec: 533.327
INFO:tensorflow:loss = 74.0947, step = 8101 (0.203 sec)
INFO:tensorflow:global_step/sec: 533.325
INFO:tensorflow:loss = 86.6593, step = 8201 (0.188 sec)
INFO:tensorflow:global_step/sec: 533.299
INFO:tensorflow:loss = 108.2, step = 8301 (0.188 sec)
INFO:tensorflow:global_step/sec: 533.357
INFO:tensorflow:loss = 59.0587, step = 8401 (0.188 sec)
INFO:tensorflow:global_step/sec: 457.136
INFO:tensorflow:loss = 95.4792, step = 8501 (0.219 sec)
INFO:tensorflow:global_step/sec: 492.301
INFO:tensorflow:loss = 162.364, step = 8601 (0.188 sec)
INFO:tensorflow:global_step/sec: 492.308
INFO:tensorflow:loss = 76.2964, step = 8701 (0.203 sec)
INFO:tensorflow:global_step/sec: 533.292
INFO:tensorflow:loss = 72.0095, step = 8801 (0.203 sec)
INFO:tensorflow:global_step/sec: 533.356
INFO:tensorflow:loss = 102.555, step = 8901 (0.172 sec)
INFO:tensorflow:global_step/sec: 533.33
INFO:tensorflow:loss = 77.1585, step = 9001 (0.188 sec)
INFO:tensorflow:global_step/sec: 

INFO:tensorflow:loss = 99.1277, step = 16501 (0.187 sec)
INFO:tensorflow:global_step/sec: 533.325
INFO:tensorflow:loss = 68.5676, step = 16601 (0.188 sec)
INFO:tensorflow:global_step/sec: 457.11
INFO:tensorflow:loss = 93.0625, step = 16701 (0.234 sec)
INFO:tensorflow:global_step/sec: 533.33
INFO:tensorflow:loss = 90.1681, step = 16801 (0.172 sec)
INFO:tensorflow:global_step/sec: 533.327
INFO:tensorflow:loss = 49.0743, step = 16901 (0.188 sec)
INFO:tensorflow:global_step/sec: 457.146
INFO:tensorflow:loss = 115.21, step = 17001 (0.219 sec)
INFO:tensorflow:global_step/sec: 533.351
INFO:tensorflow:loss = 99.9891, step = 17101 (0.187 sec)
INFO:tensorflow:global_step/sec: 492.305
INFO:tensorflow:loss = 83.7094, step = 17201 (0.203 sec)
INFO:tensorflow:global_step/sec: 533.295
INFO:tensorflow:loss = 110.424, step = 17301 (0.188 sec)
INFO:tensorflow:global_step/sec: 581.81
INFO:tensorflow:loss = 137.508, step = 17401 (0.187 sec)
INFO:tensorflow:global_step/sec: 457.166
INFO:tensorflow:loss = 1

INFO:tensorflow:loss = 102.716, step = 24901 (0.188 sec)
INFO:tensorflow:global_step/sec: 533.367
INFO:tensorflow:loss = 97.2334, step = 25001 (0.187 sec)
INFO:tensorflow:global_step/sec: 581.802
INFO:tensorflow:loss = 71.8201, step = 25101 (0.188 sec)
INFO:tensorflow:global_step/sec: 533.295
INFO:tensorflow:loss = 63.4923, step = 25201 (0.188 sec)
INFO:tensorflow:global_step/sec: 533.329
INFO:tensorflow:loss = 77.078, step = 25301 (0.172 sec)
INFO:tensorflow:global_step/sec: 533.328
INFO:tensorflow:loss = 127.869, step = 25401 (0.188 sec)
INFO:tensorflow:global_step/sec: 533.364
INFO:tensorflow:loss = 74.0746, step = 25501 (0.187 sec)
INFO:tensorflow:global_step/sec: 492.301
INFO:tensorflow:loss = 77.2204, step = 25601 (0.203 sec)
INFO:tensorflow:global_step/sec: 457.118
INFO:tensorflow:loss = 57.7805, step = 25701 (0.219 sec)
INFO:tensorflow:global_step/sec: 533.356
INFO:tensorflow:loss = 85.312, step = 25801 (0.187 sec)
INFO:tensorflow:global_step/sec: 533.294
INFO:tensorflow:loss =

INFO:tensorflow:loss = 68.5901, step = 33301 (0.187 sec)
INFO:tensorflow:global_step/sec: 492.271
INFO:tensorflow:loss = 96.4129, step = 33401 (0.203 sec)
INFO:tensorflow:global_step/sec: 533.332
INFO:tensorflow:loss = 57.9828, step = 33501 (0.203 sec)
INFO:tensorflow:global_step/sec: 492.33
INFO:tensorflow:loss = 71.8254, step = 33601 (0.188 sec)
INFO:tensorflow:global_step/sec: 492.308
INFO:tensorflow:loss = 59.0278, step = 33701 (0.219 sec)
INFO:tensorflow:global_step/sec: 533.296
INFO:tensorflow:loss = 110.423, step = 33801 (0.187 sec)
INFO:tensorflow:global_step/sec: 492.328
INFO:tensorflow:loss = 93.6677, step = 33901 (0.188 sec)
INFO:tensorflow:global_step/sec: 533.326
INFO:tensorflow:loss = 75.5287, step = 34001 (0.188 sec)
INFO:tensorflow:global_step/sec: 533.33
INFO:tensorflow:loss = 127.275, step = 34101 (0.188 sec)
INFO:tensorflow:global_step/sec: 533.328
INFO:tensorflow:loss = 94.7764, step = 34201 (0.188 sec)
INFO:tensorflow:global_step/sec: 533.326
INFO:tensorflow:loss =

In [2]:
%run regressor.py

train size:  66  test size:  5
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'temp', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from temp\model.ckpt-40000
INFO:tensorflow:Saving checkpoints for 40001 into temp\model.ckpt.
INFO:tensorflow:loss = 309.442, step = 40001
INFO:tensorflow:global_step/sec: 400.02
INFO:tensorflow:loss = 15.8379, step = 40101 (0.250 sec)
INFO:tensorflow:global_step/sec: 492.3
INFO:tensorflow:loss = 14.4023, step = 40201 (0.219 sec)
INFO:tensorflow:global_step/sec: 492.337
INFO:tensorflow:loss = 13.5335, step = 40301 (0.187 sec)
INFO:tensorflow:global_step/sec: 533.328
INFO:tensorflow:loss = 13.51, step = 40401 (0.188 sec)
INFO:tensorflow:global_step/sec: 5

INFO:tensorflow:global_step/sec: 399.996
INFO:tensorflow:loss = 11.2481, step = 47901 (0.250 sec)
INFO:tensorflow:global_step/sec: 492.301
INFO:tensorflow:loss = 10.6532, step = 48001 (0.203 sec)
INFO:tensorflow:global_step/sec: 457.137
INFO:tensorflow:loss = 10.349, step = 48101 (0.234 sec)
INFO:tensorflow:global_step/sec: 399.996
INFO:tensorflow:loss = 11.1223, step = 48201 (0.234 sec)
INFO:tensorflow:global_step/sec: 319.995
INFO:tensorflow:loss = 11.7669, step = 48301 (0.313 sec)
INFO:tensorflow:global_step/sec: 426.647
INFO:tensorflow:loss = 12.413, step = 48401 (0.250 sec)
INFO:tensorflow:global_step/sec: 400.012
INFO:tensorflow:loss = 11.9827, step = 48501 (0.250 sec)
INFO:tensorflow:global_step/sec: 457.138
INFO:tensorflow:loss = 11.2363, step = 48601 (0.203 sec)
INFO:tensorflow:global_step/sec: 492.297
INFO:tensorflow:loss = 12.0293, step = 48701 (0.203 sec)
INFO:tensorflow:global_step/sec: 492.304
INFO:tensorflow:loss = 10.4654, step = 48801 (0.219 sec)
INFO:tensorflow:global

INFO:tensorflow:global_step/sec: 426.664
INFO:tensorflow:loss = 13.441, step = 56301 (0.234 sec)
INFO:tensorflow:global_step/sec: 457.108
INFO:tensorflow:loss = 12.723, step = 56401 (0.219 sec)
INFO:tensorflow:global_step/sec: 457.137
INFO:tensorflow:loss = 10.7862, step = 56501 (0.219 sec)
INFO:tensorflow:global_step/sec: 457.165
INFO:tensorflow:loss = 11.1194, step = 56601 (0.219 sec)
INFO:tensorflow:global_step/sec: 457.139
INFO:tensorflow:loss = 10.2527, step = 56701 (0.219 sec)
INFO:tensorflow:global_step/sec: 426.66
INFO:tensorflow:loss = 11.807, step = 56801 (0.234 sec)
INFO:tensorflow:global_step/sec: 399.997
INFO:tensorflow:loss = 11.6086, step = 56901 (0.250 sec)
INFO:tensorflow:global_step/sec: 336.84
INFO:tensorflow:loss = 10.867, step = 57001 (0.313 sec)
INFO:tensorflow:global_step/sec: 355.552
INFO:tensorflow:loss = 12.094, step = 57101 (0.266 sec)
INFO:tensorflow:global_step/sec: 376.467
INFO:tensorflow:loss = 10.4711, step = 57201 (0.281 sec)
INFO:tensorflow:global_step

INFO:tensorflow:global_step/sec: 376.467
INFO:tensorflow:loss = 13.3185, step = 64701 (0.250 sec)
INFO:tensorflow:global_step/sec: 426.663
INFO:tensorflow:loss = 10.2517, step = 64801 (0.234 sec)
INFO:tensorflow:global_step/sec: 457.142
INFO:tensorflow:loss = 11.2118, step = 64901 (0.219 sec)
INFO:tensorflow:global_step/sec: 492.278
INFO:tensorflow:loss = 10.8458, step = 65001 (0.219 sec)
INFO:tensorflow:global_step/sec: 457.13
INFO:tensorflow:loss = 11.2955, step = 65101 (0.203 sec)
INFO:tensorflow:global_step/sec: 426.687
INFO:tensorflow:loss = 11.6481, step = 65201 (0.234 sec)
INFO:tensorflow:global_step/sec: 492.304
INFO:tensorflow:loss = 10.7825, step = 65301 (0.219 sec)
INFO:tensorflow:global_step/sec: 457.107
INFO:tensorflow:loss = 10.5654, step = 65401 (0.219 sec)
INFO:tensorflow:global_step/sec: 457.164
INFO:tensorflow:loss = 10.5856, step = 65501 (0.203 sec)
INFO:tensorflow:global_step/sec: 492.307
INFO:tensorflow:loss = 10.4794, step = 65601 (0.219 sec)
INFO:tensorflow:globa

INFO:tensorflow:global_step/sec: 457.169
INFO:tensorflow:loss = 10.5497, step = 73101 (0.234 sec)
INFO:tensorflow:global_step/sec: 457.11
INFO:tensorflow:loss = 11.532, step = 73201 (0.219 sec)
INFO:tensorflow:global_step/sec: 426.687
INFO:tensorflow:loss = 12.4416, step = 73301 (0.219 sec)
INFO:tensorflow:global_step/sec: 399.995
INFO:tensorflow:loss = 11.8004, step = 73401 (0.250 sec)
INFO:tensorflow:global_step/sec: 426.658
INFO:tensorflow:loss = 9.14322, step = 73501 (0.234 sec)
INFO:tensorflow:global_step/sec: 426.663
INFO:tensorflow:loss = 10.3131, step = 73601 (0.234 sec)
INFO:tensorflow:global_step/sec: 426.662
INFO:tensorflow:loss = 11.5104, step = 73701 (0.234 sec)
INFO:tensorflow:global_step/sec: 399.999
INFO:tensorflow:loss = 11.0367, step = 73801 (0.266 sec)
INFO:tensorflow:global_step/sec: 304.758
INFO:tensorflow:loss = 10.7536, step = 73901 (0.313 sec)
INFO:tensorflow:global_step/sec: 355.551
INFO:tensorflow:loss = 12.1099, step = 74001 (0.281 sec)
INFO:tensorflow:global

In [3]:
%run regressor.py

train size:  17  test size:  5
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'temp', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from temp\model.ckpt-80000
INFO:tensorflow:Saving checkpoints for 80001 into temp\model.ckpt.
INFO:tensorflow:loss = 208.231, step = 80001
INFO:tensorflow:global_step/sec: 492.396
INFO:tensorflow:loss = 4.38787, step = 80101 (0.203 sec)
INFO:tensorflow:global_step/sec: 457.138
INFO:tensorflow:loss = 3.96589, step = 80201 (0.203 sec)
INFO:tensorflow:global_step/sec: 492.302
INFO:tensorflow:loss = 3.75269, step = 80301 (0.203 sec)
INFO:tensorflow:global_step/sec: 533.325
INFO:tensorflow:loss = 3.743, step = 80401 (0.188 sec)
INFO:tensorflow:global_step/sec

INFO:tensorflow:global_step/sec: 427.065
INFO:tensorflow:loss = 3.53668, step = 87901 (0.235 sec)
INFO:tensorflow:global_step/sec: 349.391
INFO:tensorflow:loss = 3.552, step = 88001 (0.284 sec)
INFO:tensorflow:global_step/sec: 414.533
INFO:tensorflow:loss = 3.42179, step = 88101 (0.237 sec)
INFO:tensorflow:global_step/sec: 457.109
INFO:tensorflow:loss = 3.56203, step = 88201 (0.234 sec)
INFO:tensorflow:global_step/sec: 426.688
INFO:tensorflow:loss = 3.35812, step = 88301 (0.219 sec)
INFO:tensorflow:global_step/sec: 457.115
INFO:tensorflow:loss = 3.54249, step = 88401 (0.234 sec)
INFO:tensorflow:global_step/sec: 426.657
INFO:tensorflow:loss = 3.72012, step = 88501 (0.219 sec)
INFO:tensorflow:global_step/sec: 457.167
INFO:tensorflow:loss = 3.4616, step = 88601 (0.234 sec)
INFO:tensorflow:global_step/sec: 457.109
INFO:tensorflow:loss = 3.31303, step = 88701 (0.219 sec)
INFO:tensorflow:global_step/sec: 457.167
INFO:tensorflow:loss = 3.58373, step = 88801 (0.219 sec)
INFO:tensorflow:global_

INFO:tensorflow:global_step/sec: 426.667
INFO:tensorflow:loss = 3.21587, step = 96301 (0.219 sec)
INFO:tensorflow:global_step/sec: 457.139
INFO:tensorflow:loss = 3.24115, step = 96401 (0.219 sec)
INFO:tensorflow:global_step/sec: 457.137
INFO:tensorflow:loss = 3.33886, step = 96501 (0.219 sec)
INFO:tensorflow:global_step/sec: 492.307
INFO:tensorflow:loss = 3.37249, step = 96601 (0.219 sec)
INFO:tensorflow:global_step/sec: 457.108
INFO:tensorflow:loss = 3.19156, step = 96701 (0.219 sec)
INFO:tensorflow:global_step/sec: 426.684
INFO:tensorflow:loss = 3.1639, step = 96801 (0.219 sec)
INFO:tensorflow:global_step/sec: 457.111
INFO:tensorflow:loss = 3.47614, step = 96901 (0.219 sec)
INFO:tensorflow:global_step/sec: 457.164
INFO:tensorflow:loss = 3.24246, step = 97001 (0.234 sec)
INFO:tensorflow:global_step/sec: 426.64
INFO:tensorflow:loss = 3.25332, step = 97101 (0.234 sec)
INFO:tensorflow:global_step/sec: 426.661
INFO:tensorflow:loss = 3.249, step = 97201 (0.219 sec)
INFO:tensorflow:global_s

INFO:tensorflow:loss = 3.00975, step = 104601 (0.219 sec)
INFO:tensorflow:global_step/sec: 457.166
INFO:tensorflow:loss = 3.03236, step = 104701 (0.219 sec)
INFO:tensorflow:global_step/sec: 492.271
INFO:tensorflow:loss = 2.93071, step = 104801 (0.203 sec)
INFO:tensorflow:global_step/sec: 492.302
INFO:tensorflow:loss = 3.20982, step = 104901 (0.219 sec)
INFO:tensorflow:global_step/sec: 457.168
INFO:tensorflow:loss = 3.21428, step = 105001 (0.203 sec)
INFO:tensorflow:global_step/sec: 457.107
INFO:tensorflow:loss = 3.19003, step = 105101 (0.219 sec)
INFO:tensorflow:global_step/sec: 492.336
INFO:tensorflow:loss = 3.18813, step = 105201 (0.219 sec)
INFO:tensorflow:global_step/sec: 457.11
INFO:tensorflow:loss = 3.00472, step = 105301 (0.203 sec)
INFO:tensorflow:global_step/sec: 426.688
INFO:tensorflow:loss = 3.20005, step = 105401 (0.234 sec)
INFO:tensorflow:global_step/sec: 492.302
INFO:tensorflow:loss = 3.11558, step = 105501 (0.219 sec)
INFO:tensorflow:global_step/sec: 426.653
INFO:tensor

INFO:tensorflow:loss = 3.25742, step = 112901 (0.219 sec)
INFO:tensorflow:global_step/sec: 492.299
INFO:tensorflow:loss = 3.13876, step = 113001 (0.219 sec)
INFO:tensorflow:global_step/sec: 457.141
INFO:tensorflow:loss = 2.80604, step = 113101 (0.203 sec)
INFO:tensorflow:global_step/sec: 492.268
INFO:tensorflow:loss = 3.1073, step = 113201 (0.219 sec)
INFO:tensorflow:global_step/sec: 457.137
INFO:tensorflow:loss = 2.87249, step = 113301 (0.203 sec)
INFO:tensorflow:global_step/sec: 426.688
INFO:tensorflow:loss = 3.0152, step = 113401 (0.234 sec)
INFO:tensorflow:global_step/sec: 492.269
INFO:tensorflow:loss = 2.84994, step = 113501 (0.219 sec)
INFO:tensorflow:global_step/sec: 457.162
INFO:tensorflow:loss = 2.89197, step = 113601 (0.203 sec)
INFO:tensorflow:global_step/sec: 457.143
INFO:tensorflow:loss = 2.92032, step = 113701 (0.234 sec)
INFO:tensorflow:global_step/sec: 457.106
INFO:tensorflow:loss = 2.90763, step = 113801 (0.203 sec)
INFO:tensorflow:global_step/sec: 492.306
INFO:tensorf